In [23]:
import psycopg2 as pg
from datetime import datetime
from pprint import pprint

PARAMS = {'database':'netologiaveronika', 
        'user':'netologiaveronika', 'password':'lehfrb', 'host':'pg.codecontrol.ru', 'port':59432}


student = {'name': 'bob', 'gpa': 10.6, 'birth': '1985-07-31'}



def create_db():  # создает таблицы
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        # cur.execute('''drop table Student;
        # drop table Course''')
        cur.execute('''
            create table if not exists Student(student_id serial primary key not null,
            name varchar(100) not null,
            gpa numeric(10,3) null,
            birth timestamp with time zone null);
        
        create table if not exists Course(course_id serial primary key not null,
        course_name varchar(100) not null);''')

        cur.execute("""create table if not exists student_course (
                id serial primary key,
                student_id integer references Student(student_id),
                course_id integer references  Course(course_id));
                """)
        # cur.execute('''
        # select * from Student left join Course on Student.student_id = Course.course_id''')
create_db()

def create_course():
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute("""
        insert into Course(course_id, course_name) values(%s, %s);
        """, ('1', 'Python'))
#create_course()

def add_student(student):  # просто создает студента
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute("""
        insert into Student(name, gpa, birth) values(%s, %s, %s);
        """, (student.get('name'), student.get('gpa'), student.get('birth')))


def get_student(entered_id):
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute('''
        select * from Student where student_id=%s''', (entered_id,))
    print(cur.fetchone())


def get_students(course_id):  # возвращает студентов определенного курса
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute('''
        select name, student_id from Student
        inner join Course
        on Student.student_id = Course.course_id
        where course_id=%s;''', (course_id,))
    print(cur.fetchall())


def add_students(course_id, students):  # создает студентов и записывает их на курс
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        if course_id:
            for data in students:
                cur.execute("insert into Student(name, gpa, birth) values (%s, %s, %s) returning student_id;",
                            (data.get('name'), data.get('gpa'), data.get('birth')))
                students_id = cur.fetchone()
                cur.execute("insert into student_course (student_id, course_id) values (%s, %s)",
                            (students_id, course_id))
        else:
            print("It seems we don't have such course:(")


def get_all():
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute('''
        select * from Student''')
    pprint(cur.fetchall())


if __name__ == '__main__':
    # create_course()

    student = {'name': 'bob', 'gpa': 5.56, 'birth': '1988-01-31'}
    #add_student(student)
    get_all()
    # get_student(9)
    students = [{'name': 'bob', 'gpa': 5.33, 'birth': '1985-01-31'},
                {'name': 'barbara', 'gpa': 6.66, 'birth': '1990-05-11'},
                {'name': 'frank', 'gpa': 4.15, 'birth': '1985-12-11'},
                ]
    add_students(2, students)
    get_students()

[(2,
  'bob',
  Decimal('5.330'),
  datetime.datetime(1985, 1, 31, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))),
 (3,
  'barbara',
  Decimal('6.660'),
  datetime.datetime(1990, 5, 11, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))),
 (4,
  'frank',
  Decimal('4.150'),
  datetime.datetime(1985, 12, 11, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)))]


ForeignKeyViolation: insert or update on table "student_course" violates foreign key constraint "student_course_course_id_fkey"
DETAIL:  Key (course_id)=(2) is not present in table "course".


In [16]:
def create_course():
    with pg.connect(database='netologiaveronika', user='netologiaveronika', password='lehfrb', host='pg.codecontrol.ru', port=59432) as connect:
        cur = connect.cursor()
        cur.execute("""
        insert into Course(course_id, course_name) values(%s, %s);
        """, ('1', 'Python'))
create_course()

UndefinedTable: relation "course" does not exist
LINE 2:         insert into Course(course_id, course_name) values('1...
                            ^
